In [9]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import pandas as pd

import os

from datetime import datetime, timedelta, date


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [10]:
#### Setting up strava API client

#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

## Enable accessing private activities
auth_url = client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)

strava_client = stravalib.client.Client(access_token=access_token)
athlete = strava_client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))

athlete name Brian, athlete id 3360678.


In [11]:
## Set up google sheets client, open worksheet
import pygsheets

gc = pygsheets.authorize(outh_file='client_secret.json', no_cache=True)

# Open spreadsheet and then workseet
sh = gc.open('Milburn Ultimate Scores')

In [12]:
## Get last entry from Raw Point Spreadsheet

wks = sh.worksheet_by_title('raw_points')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in wks.get_col(2) if d != '' and d != 'Start Time']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]

# lap_start_date = lap_start_date - timedelta(days=5)

lap_start_date

datetime.datetime(2018, 3, 24, 13, 17, 30)

In [13]:
def get_strava_description(activity, p=False):
    new_activity = strava_client.get_activity(activity.id)
    try:
        scores, color = new_activity.description.split(' ')
        try:
            team_score, opponent_score = scores.split('-')
        except ValueError:
            team_score, opponent_score, color = None, None, None     
    except ValueError:
        scores = None
        team_score, opponent_score, color = None, None, None           

    try:
        color = color.lower()
    except AttributeError:
        color = None
        
    if p:
        print(new_activity.start_date)
        print(new_activity.description)
        print(team_score, opponent_score, color)
        
    return team_score, opponent_score, color

In [14]:
## Functions
def extract_events(run):
    lap_nums = []
    start_times = []
    elapsed_times = []
    for l in run.laps:
        try:
            lap_nums.append(int(l['name'].split(' ')[-1]))
            start_times.append(datetime.strptime(l['start_date_local'], '%Y-%m-%dT%H:%M:%SZ'))
            elapsed_times.append(timedelta(seconds=l['elapsed_time']))
        except TypeError:
            lap_nums.append(int(l.name.split(' ')[-1]))
            start_times.append(l.start_date_local)
            elapsed_times.append(l.elapsed_time)

    lap_nums = np.array(lap_nums)
    
    events = []
    for n, s, e in zip(np.diff(lap_nums), start_times, elapsed_times):
        events.append([n, s, e])
    
    return (events)
        
def process_events(events):
    
    event_lookup = [
        '',
        'team_point',
        'opponent_point',
        'my_point',
        'game',
        'game',
        'game'
    ]
    
    games = []
    game_num = 0
    added = False

    base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0, 'events':[], 'start_time':None, 'end_time':None}
    game = base_game

    for event in events:
        if game['start_time'] is None:
            game['start_time'] = event[1]

        game['end_time'] = event[1] + event[2]

        event_type = event_lookup[event[0]]
        game['events'].append((event_type, event[1], event[2]))


        if event_type == 'game':
            games.append(game)
            game_num += 1
            game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num, 'events':[], 'start_time':None, 'end_time':None}
            added = True

        elif event_type == 'my_point':
            game[event_type] += 1
            game['team_point'] += 1
            added = False

        else:
            game[event_type] += 1
            added = False


    if not added:
        games.append(game)
    
    
    ## Assign game winners
    for game in games:
        if game['team_point'] > game['opponent_point']:
            game['win'] = True
        else:
            game['win'] = False
            
    return games

In [15]:
## Compile raw points for export
all_data = []

runs = []
for activity in strava_client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        a = strava_client.get_activity(activity.id)
        runs.append(a)
        
for run in reversed(runs):
   
    events = extract_events(run)

    point_df = pd.DataFrame(events, columns=['count', 'start_time', 'elapsed_time'])
    point_df['start_time'] = point_df['start_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    point_df['elapsed_time'] = point_df['elapsed_time'].apply(lambda x: x.seconds)
    
    data = point_df.sort_index(ascending=False).as_matrix().tolist()
    all_data = all_data + data    

In [16]:
## Write raw points to gsheets
wks = sh.worksheet_by_title('raw_points')
wks.insert_rows(2, values=all_data, number=len(all_data))

In [17]:
## Get last entry from Data Spreadsheet

wks = sh.worksheet_by_title('game_summaries')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d') for d in wks.get_col(1) if d != '' and d != 'Date']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]
lap_start_date
# from datetime import timedelta
# lap_start_date = lap_start_date - timedelta(days=2)

datetime.datetime(2018, 3, 24, 0, 0)

In [18]:
wks = sh.worksheet_by_title('raw_points')
col_names = wks.get_all_values()[1][0:3]

val_lists = wks.get_all_values()[2:]
val_lists = [v[0:3] for v in val_lists]
processed_raw_points = pd.DataFrame(val_lists, columns=col_names)

# Process
processed_raw_points['Type'] = processed_raw_points['Type'].apply(lambda x: int(x))

processed_raw_points['Start Time'] = processed_raw_points['Start Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
processed_raw_points['Day'] = processed_raw_points['Start Time'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))
processed_raw_points['Elapsed Time (Sec)'] = processed_raw_points['Elapsed Time (Sec)'].apply(lambda x: timedelta(seconds=int(x)))


processed_raw_points = processed_raw_points[processed_raw_points['Start Time'] > lap_start_date].sort_values('Start Time', ascending=True).reset_index()
processed_raw_points

,index,Type,Start Time,Elapsed Time (Sec),Day
0,21,2,2018-03-26 12:16:53,00:00:11,2018-03-26
1,20,2,2018-03-26 12:17:09,00:00:21,2018-03-26
2,19,2,2018-03-26 12:17:29,00:01:54,2018-03-26
3,18,2,2018-03-26 12:19:25,00:01:29,2018-03-26
4,17,3,2018-03-26 12:20:55,00:03:18,2018-03-26
5,16,2,2018-03-26 12:24:16,00:01:16,2018-03-26
6,15,4,2018-03-26 12:25:36,00:00:22,2018-03-26
7,14,2,2018-03-26 12:25:59,00:08:22,2018-03-26
8,13,2,2018-03-26 12:34:24,00:05:24,2018-03-26
9,12,1,2018-03-26 12:39:49,00:01:41,2018-03-26


In [19]:
games = []

runs = []
for activity in strava_client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        a = strava_client.get_activity(activity.id)
        runs.append(a)
        
for day, pdf in processed_raw_points.groupby('Day', sort=False):
    print(day)

    
    team_wins, opponent_wins, color = get_strava_description(runs[0], p=True)

    events = pdf[['Type', 'Start Time', 'Elapsed Time (Sec)']].as_matrix().tolist()
    
    for e in events:
        print(e)
    current_games = process_events(events)
    
    for g in current_games:
        g['my_color'] = color
        g['team_wins'] = team_wins
        g['opponent_wins'] = opponent_wins
        

    games = games + current_games


2018-03-26 00:00:00
2018-03-26 17:16:53+00:00
2-1 white
2 1 white
[2, Timestamp('2018-03-26 12:16:53'), Timedelta('0 days 00:00:11')]
[2, Timestamp('2018-03-26 12:17:09'), Timedelta('0 days 00:00:21')]
[2, Timestamp('2018-03-26 12:17:29'), Timedelta('0 days 00:01:54')]
[2, Timestamp('2018-03-26 12:19:25'), Timedelta('0 days 00:01:29')]
[3, Timestamp('2018-03-26 12:20:55'), Timedelta('0 days 00:03:18')]
[2, Timestamp('2018-03-26 12:24:16'), Timedelta('0 days 00:01:16')]
[4, Timestamp('2018-03-26 12:25:36'), Timedelta('0 days 00:00:22')]
[2, Timestamp('2018-03-26 12:25:59'), Timedelta('0 days 00:08:22')]
[2, Timestamp('2018-03-26 12:34:24'), Timedelta('0 days 00:05:24')]
[1, Timestamp('2018-03-26 12:39:49'), Timedelta('0 days 00:01:41')]
[1, Timestamp('2018-03-26 12:41:31'), Timedelta('0 days 00:03:44')]
[2, Timestamp('2018-03-26 12:45:18'), Timedelta('0 days 00:01:28')]
[2, Timestamp('2018-03-26 12:46:44'), Timedelta('0 days 00:01:52')]
[3, Timestamp('2018-03-26 12:48:40'), Timedelta('0

In [20]:
df = pd.DataFrame(games).dropna()
df

,end_time,events,game_num,my_color,my_point,opponent_point,opponent_wins,start_time,team_point,team_wins,win
0,2018-03-26 12:25:58,"[(opponent_point, 2018-03-26 12:16:53, 0 days ...",0,white,1,5,1,2018-03-26 12:16:53,1,2,False
1,2018-03-26 12:54:21,"[(opponent_point, 2018-03-26 12:25:59, 0 days ...",1,white,1,4,1,2018-03-26 12:25:59,5,2,True
2,2018-03-26 13:00:12,"[(team_point, 2018-03-26 12:54:30, 0 days 00:0...",2,white,1,1,1,2018-03-26 12:54:30,3,2,True


In [21]:
df['date'] = df.end_time.apply(lambda x: datetime(x.year, x.month, x.day))

df = df.set_index(['date', 'game_num'], drop=False)
df

end_time  \
date       game_num                       
2018-03-26 0        2018-03-26 12:25:58   
           1        2018-03-26 12:54:21   
           2        2018-03-26 13:00:12   

                                                                events  \
date       game_num                                                      
2018-03-26 0         [(opponent_point, 2018-03-26 12:16:53, 0 days ...   
           1         [(opponent_point, 2018-03-26 12:25:59, 0 days ...   
           2         [(team_point, 2018-03-26 12:54:30, 0 days 00:0...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-26 0                0    white         1               5   
           1                1    white         1               4   
           2                2    white         1               1   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-26 0                    1 2018-03-26 12:16:53           1         2   
           1                    1 2018-03-26 12:25:59           5         2   
           2                    1 2018-03-26 12:54:30           3         2   

                       win       date  
date       game_num                    
2018-03-26 0         False 2018-03-26  
           1          True 2018-03-26  
           2          True 2018-03-26

In [22]:
pdf = df

In [23]:
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None
pdf['game_winner'] = None

In [24]:
for (date, game_num), row in pdf.iterrows():      
    
    if row.my_color == 'white':
        if row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'opponent_wins']            
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'opponent_point']

    else:
        if not row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'opponent_wins']    
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'opponent_point']


In [25]:
pdf

end_time  \
date       game_num                       
2018-03-26 0        2018-03-26 12:25:58   
           1        2018-03-26 12:54:21   
           2        2018-03-26 13:00:12   

                                                                events  \
date       game_num                                                      
2018-03-26 0         [(opponent_point, 2018-03-26 12:16:53, 0 days ...   
           1         [(opponent_point, 2018-03-26 12:25:59, 0 days ...   
           2         [(team_point, 2018-03-26 12:54:30, 0 days 00:0...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-26 0                0    white         1               5   
           1                1    white         1               4   
           2                2    white         1               1   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-26 0                    1 2018-03-26 12:16:53           1         2   
           1                    1 2018-03-26 12:25:59           5         2   
           2                    1 2018-03-26 12:54:30           3         2   

                       win       date white_wins color_wins  white_point  \
date       game_num                                                        
2018-03-26 0         False 2018-03-26          2          1            1   
           1          True 2018-03-26          2          1            5   
           2          True 2018-03-26          2          1            3   

                     color_point game_winner  
date       game_num                           
2018-03-26 0                   5       Color  
           1                   4       White  
           2                   1       White

In [26]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [27]:
scores = []
for (date, game_num), game in pdf.iterrows():
    base_dict = dict(date=date, end_time=game.end_time, game_num=game_num, white_wins=game.white_wins, color_wins=game.color_wins, game_winner=game.game_winner)    
    scores.append(merge_two_dicts(base_dict, dict(team='white', team_score=game.white_point, my_score=game.my_point if game.my_color == 'white' else None)))
    scores.append(merge_two_dicts(base_dict, dict(team='color', team_score=game.color_point, my_score=game.my_point if game.my_color == 'colors' else None)))

score_df = pd.DataFrame(scores).set_index(['date', 'game_num', 'team'], drop=False).sort_values('end_time')

In [28]:
score_df

color_wins       date            end_time  game_num  \
date       game_num team                                                        
2018-03-26 0        white          1 2018-03-26 2018-03-26 12:25:58         0   
                    color          1 2018-03-26 2018-03-26 12:25:58         0   
           1        white          1 2018-03-26 2018-03-26 12:54:21         1   
                    color          1 2018-03-26 2018-03-26 12:54:21         1   
           2        white          1 2018-03-26 2018-03-26 13:00:12         2   
                    color          1 2018-03-26 2018-03-26 13:00:12         2   

                          game_winner  my_score   team  team_score white_wins  
date       game_num team                                                       
2018-03-26 0        white       Color       1.0  white           1          2  
                    color       Color       NaN  color           5          2  
           1        white       White       1.0  white           5          2  
                    color       White       NaN  color           4          2  
           2        white       White       1.0  white           3          2  
                    color       White       NaN  color           1          2

In [29]:
out_df = score_df[['date', 'white_wins', 'color_wins', 'game_num', 'game_winner', 'team', 'team_score', 'my_score']].fillna(value='')
out_df['date'] = out_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [30]:
out_df

date white_wins color_wins  game_num  \
date       game_num team                                                
2018-03-26 0        white  2018-03-26          2          1         0   
                    color  2018-03-26          2          1         0   
           1        white  2018-03-26          2          1         1   
                    color  2018-03-26          2          1         1   
           2        white  2018-03-26          2          1         2   
                    color  2018-03-26          2          1         2   

                          game_winner   team  team_score my_score  
date       game_num team                                           
2018-03-26 0        white       Color  white           1        1  
                    color       Color  color           5           
           1        white       White  white           5        1  
                    color       White  color           4           
           2        white       White  white           3        1  
                    color       White  color           1

In [31]:
data = out_df.as_matrix().tolist()
wks = sh.worksheet_by_title('game_summaries')
wks.insert_rows(2, values=data, number=len(data))